A sample CBF design is saved in a csv file. We first read in the data

In [1]:
import pandas as pd
test_cbf = pd.read_csv('cbf.csv', index_col=0, header=0).T
cbf_params = test_cbf.iloc[0]
# preview the data
cbf_params

num_bays           5.000000
num_stories        4.000000
num_frames         2.000000
S_1                1.185500
T_m                2.749952
k_ratio            5.739507
Q                  0.071810
moat_ampli         1.775365
RI                 1.445491
L_bay             30.000000
h_story           13.000000
S_s                2.281500
W              10687.500000
W_s             8437.500000
mu_1               0.023457
mu_2               0.086961
R_1               12.374762
R_2               51.674927
T_e                2.830994
k_e                0.012748
zeta_e             0.147725
D_m               23.755869
Name: 24, dtype: float64

In [2]:
# temporary fix to add loads to the Series
import numpy as np
cbf_params['w_fl'] = np.array([3., 3., 3., 3., 2.013])
cbf_params['P_lc'] = np.array([1800., 1800., 1800., 1800., 1208.])

# accessing a sample value
n_bays = int(cbf_params['num_bays'])
cbf_params

num_bays                                            5.0
num_stories                                         4.0
num_frames                                          2.0
S_1                                              1.1855
T_m                                            2.749952
k_ratio                                        5.739507
Q                                               0.07181
moat_ampli                                     1.775365
RI                                             1.445491
L_bay                                              30.0
h_story                                            13.0
S_s                                              2.2815
W                                               10687.5
W_s                                              8437.5
mu_1                                           0.023457
mu_2                                           0.086961
R_1                                           12.374762
R_2                                           51

# Building class

It's advantageous to make a class for our data, which allows us to internally store all of the design parameters and objects of the models (nodes, elements) without having to pass long lists of arguments to each function. We can also store all functions related to the building as methods that act on the Building object itself.

For example, the number_nodes method acts on the Building object by taking the Building and finding its number of bays and stories in order to add the lists of nodes and elements to the Building. Later on, we will add model and analyze as additional methods to the Building.

In [18]:
class Building:
    
    #########################################
    # INITIALIZING
    #########################################
    
    # import attributes as building characteristics from pd.Series
    
    # now the Series you see above is converted into a Building object
    # Values in the series are now attributes of the object
    def __init__(self, design):
        for key, value in design.items():
            setattr(self, key, value)
            
    #########################################
    # NUMBERING NODES AND ELEMENTS
    #########################################
    
    def number_nodes(self):
        
        # to interact with the attributes use the following below
        n_bays = int(self.num_bays)
        n_stories = int(self.num_stories)
        
        n_braces = 2 # for now
        
        # larger than 8 bays is not supported
        assert n_bays < 9
        
        ###### Main node system ######
        # Fixed nodes are 8xx, with xx being sequential from leftmost base
        #   898 and 899 are reserved for wall
        
        # All nodes are numbered xy, with x indicating floor number 
        #   1 at ground, n_stories+1 at roof
        # and y indicating column line
        #   0 at leftmost, n_bays at rightmost
        
        # Leaning column nodes are appended to the right at the same floor (n_bay+1)
        ##############################
        
        ##############################
        # YOUR CODE BELOW
        # Write your own implementation of the node numbering system
        # Store the results for each types of nodes by replacing the empty [] lists
        ##############################
        
        #main nodes
        main_nodes = []
        for i in range(1, n_bays+1):
            for j in range(0, n_stories+1):
                main_nodes.append(str(i)+str(j))
                
        # base nodes
        base_id = 800
        base_nodes = []
        for b in range(0,n_bays+1):
            base_nodes.append("8"+"0"+str(b))
        
        # wall nodes
        wall_nodes = [898, 899]
        
        # floor and leaning column nodes
        leaning_nodes = []
        for l in range(1, n_stories+1):
            leaning_nodes.append(str(l)+str(n_bays+1))
        
        # EZ- brace nodes - point in the middle of the beam
        #TODO: Start at 2
        brace_nodes = []
        for b in range(1, n_stories+1):
            brace_nodes.append("1"+str(b)+"55")
            brace_nodes.append(str(n_bays)+str(b)+"55")
        
        ###### Spring node system ######
        # Spring support nodes have the coordinates XYA, XY being the parent node
        # A is 6,7,8,9 for S,W,N,E respectively
        ################################
        
        # flatten list to get all nodes 
        # EZ- main nodes and leaning nodes for everyfloor minus top floor
        floor_nodes = []
        for i in range(1, n_bays+2):
            for j in range(1, n_stories):
                floor_nodes.append(str(i)+str(j))
    
        
        # make south node if not on bottom floor
        
        # make west node if not on the leftmost column and bottom floor
        
        # make north node if not on top floor
        
        # make east node if not on rightmost column and bottom floor
        
        # repeat for leaning columns, only N-S
        lc_spr_nodes = []
        for s in range(len(leaning_nodes)):
            lc_spr_nodes.append(leaning_nodes[s]+"6") #south
            lc_spr_nodes.append(leaning_nodes[s]+"7") #west
            lc_spr_nodes.append(leaning_nodes[s]+"8") #north
            lc_spr_nodes.append(leaning_nodes[s]+"9") #east
        
        spring_nodes = []
        for s in range(len(main_nodes)):
            spring_nodes.append(main_nodes[s]+"6") #south
            spring_nodes.append(main_nodes[s]+"7") #west
            spring_nodes.append(main_nodes[s]+"8") #north
            spring_nodes.append(main_nodes[s]+"9") #east
        
        ###### Element system ######
        # Elements all have an ID or "series" that all numbers of that type follows
        ################################
        
        # column elements, series 100
        col_id = 100
        # make column if not the top floor
        col_elems = []
        for c in range(len(floor_nodes)):
            col_elems.append("1"+floor_nodes[c])
        
        # leaning column elements 
        lc_elems = []
        for l in range(1, n_stories):
            lc_elems.append("1"+str(l)+str(n_bays+1))
        
        # beam elements, series 200
        beam_id = 200 
        # make beam if not the last bay and not the bottom floor
        beam_elems = []
        for b in range(len(main_nodes)):
            beam_elems.append("2"+main_nodes[b])
        
        # truss elements, series 300
        truss_id = 300
        # make truss on the last bay for all floors
        truss_elems = []
        
        
        # diaphragm elements, series 400
        diaph_id = 400
        # make diaphragm if not the last bay on the bottom floor
        diaph_elems = []
        for d in range(1, n_bays+1):
            diaph_elems.append("41"+"d")
        
        # isolator elements, series 1000
        isol_id = 1000
        # make isolators above base nodes
        isol_elems = []
        for i in range(len(base_nodes)):
            isol_elems.append(isol_id+i)
        
        # spring elements, series 5000
        spring_id = 5000
        spring_elems = []
        for s in range(len(spring_nodes)):
            spring_elems.append(str(spring_id)+spring_nodes[s])
            
        lc_spr_elems = []
        for l in range(len(lc_spr_nodes)):
            lc_spr_nodes.append(str(spring_id)+lc_spr_nodes[l])
        
        # wall elements, series 8000
        wall_id = 8000
        wall_elems = [8898, 8899]
        
        # write your own implementation for braces below!
        brace_elems = []
        for i in range(len(brace_nodes)):
            brace_elems.append("3"+brace_nodes[i]+"1") #connecting bottom left
            brace_elems.append("3"+brace_nodes[i]+"2") #connecting bottom right
        
        
        ###### Storing results ######
        # It's most efficient for us to store the lists of items nested in a dictionary of that type
        # The dictionaries themselves are now stored as attributes that we can easily access later on
        
        # example
        # Later when we access the objects, we simply get all of the floor nodes by using bldg.node_tags['floor_nodes']
        # Repeat the same for beam elements, e.g., with bldg.elem_tags['beam_elems']
        ################################
        self.node_tags = {
            'base': base_nodes,
            'wall': wall_nodes,
            'floor': floor_nodes,
            'leaning': leaning_nodes,
            'spring': spring_nodes,
            'lc_spring': lc_spr_nodes
            }
        
        self.elem_tags = {
            'col': col_elems, 
            'leaning': lc_elems, 
            'beam': beam_elems,
            'truss': truss_elems, 
            'diaphragm': diaph_elems, 
            'isolator': isol_elems, 
            'spring': spring_elems, 
            'lc_spring': lc_spr_elems, 
            'wall': wall_elems
            }
        
        self.elem_ids = {
            'col': col_id, 
            'leaning': col_id, 
            'beam': beam_id,
            'truss': truss_id, 
            'diaphragm': diaph_id, 
            'isolator': isol_id, 
            'spring': spring_id, 
            'lc_spring': spring_id, 
            'wall': wall_id,
            'base': base_id
            }
        
    #########################################
    # BUILDING THE MODEL IN OPENSEES
    #########################################
    
    def model_frame(self):
         # import OpenSees and libraries
        import openseespy.opensees as ops
        
        # remove existing model
        ops.wipe()

        # units: in, kip, s
        # dimensions
        inch    = 1.0
        ft      = 12.0*inch
        sec     = 1.0
        g       = 386.4*inch/(sec**2)
        kip     = 1.0
        ksi     = kip/(inch**2)
        
        L_bay = self.L_bay * ft     # ft to in
        h_story = self.h_story * ft
        w_floor = self.w_fl / ft    # kip/ft to kip/in
        p_lc = self.P_lc
        
        # set modelbuilder
        # x = horizontal, y = in-plane, z = vertical
        # command: model('basic', '-ndm', ndm, '-ndf', ndf=ndm*(ndm+1)/2)
        ops.model('basic', '-ndm', 3, '-ndf', 6)
        
        # masses of the nodes
        m_grav_inner = w_floor * L_bay / g
        m_grav_outer = w_floor * L_bay / 2 /g
        m_lc = p_lc / g
        
        # load for isolators vertical
        p_outer = sum(w_floor)*L_bay/2
        p_inner = sum(w_floor)*L_bay
        
        # nominal change
        L_beam = L_bay
        L_col = h_story
        
        ##############################
        # YOUR CODE BELOW
        # Write your own code to place the nodes
        ##############################
        
        # base nodes
        base_nodes = self.node_tags['base']
        
        # place base nodes here
        # ops.node(node_tag, x-coord, y-coord, z-coord)
        
        # remember to fix the nodes with
        # ops.fix(node_tag, 1, 1, 1, 1, 1, 1) for fully fixed
        # ops.fix(nd, 0, 1, 0, 1, 0, 1) for fully free
        
        # wall nodes (should only be two)
        n_bays = int(self.num_bays)
        n_floors = int(self.num_stories)
        
        # place wall nodes here
        wall_nodes = self.node_tags['wall']
        
        
        # place main nodes here
        # structure nodes
        floor_nodes = self.node_tags['floor']
            
        # leaning column nodes
        leaning_nodes = self.node_tags['leaning']
        
        # SPECIAL CASE: we roller fix the bottom node of the leaning column
        # use ops.fix(nd, 0, 1, 1, 1, 0, 1)
        
        # place spring nodes here
        # spring nodes
        spring_nodes = self.node_tags['spring']
            
        # leaning column spring nodes
        lc_spr_nodes = self.node_tags['lc_spring']
            
        print('Nodes placed.')

## Test out the building

First, initialize it by getting the parameters from the Series. Then number the nodes.

In [19]:
cbf_bldg = Building(cbf_params)

cbf_bldg.number_nodes()

cbf_bldg.node_tags
cbf_bldg.elem_tags
cbf_bldg.elem_ids


{'col': 100,
 'leaning': 100,
 'beam': 200,
 'truss': 300,
 'diaphragm': 400,
 'isolator': 1000,
 'spring': 5000,
 'lc_spring': 5000,
 'wall': 8000,
 'base': 800}

## Try modeling the frame

We start with just the nodes for now.

In [20]:
cbf_bldg.model_frame()

Nodes placed.
